In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 4.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=34cd800a75dac5f5bf7cf8ac6a8d0a66172228d03bbb47bcf85e840329f30637
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=389541ea41b0048b33d0434c966efe85698ea91f5dc5b945489d58fd2df95a11
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
   keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter',min_value=32, max_value=128, step=16),
      kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
      activation='relu',
      input_shape=(28,28,1)
   ),
   keras.layers.Conv2D(
      filters=hp.Int('conv_2_filter',min_value=32, max_value=64, step=16),
      kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
      activation='relu'
   ),
   keras.layers.Flatten(),
   keras.layers.Dense(
       units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
       activation='relu'
   ),
   keras.layers.Dense(10, activation='softmax')                         
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output', project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 11m 22s]
val_accuracy: 0.85916668176651

Best val_accuracy So Far: 0.9081666469573975
Total elapsed time: 00h 51m 08s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |48                |112               
conv_1_kernel     |3                 |3                 
conv_2_filter     |32                |48                
conv_2_kernel     |3                 |5                 
dense_1_units     |112               |64                
learning_rate     |0.001             |0.001             

Epoch 1/3
 757/1688 [============>.................] - ETA: 1:18 - loss: 0.4624 - accuracy: 0.8350

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        27680     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1179712   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
=================================================================
Total params: 1,209,002
Trainable params: 1,209,002
Non-trainable params: 0